In [110]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import csv
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV

In [130]:
#Import the data and replace "" with NaN 
data=pd.read_csv("/Users/shilpasingla/Data Science/Data Scientist Case Study/Data Scientist Case Study/prob_2/sample_lead_data.csv",
                 index_col=[0],na_values='""')


In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   MkVendorID            100000 non-null  int64  
 1   SourceCode            100000 non-null  object 
 2   VendorDescription     100000 non-null  object 
 3   LeadType              94348 non-null   object 
 4   VendorInternalCode    66405 non-null   object 
 5   mkCampaignID          100000 non-null  int64  
 6   mkMarketingChannelId  100000 non-null  float64
 7   MkMarketingChannel    100000 non-null  object 
 8   Address               80760 non-null   object 
 9   City                  85221 non-null   object 
 10  State                 98843 non-null   object 
 11  Country               99168 non-null   object 
 12  Zip                   99780 non-null   object 
 13  SyCampusID            100000 non-null  int64  
 14  HighestEducation      68406 non-null   object 
 15  C

In [135]:
#original csv file has extra quotes "" in each string .We have to remove them
data2 = data.applymap(lambda x: x.strip('""') if isinstance(x, str)a else x)


In [136]:
data2.head()

,MkVendorID,SourceCode,VendorDescription,LeadType,VendorInternalCode,mkCampaignID,mkMarketingChannelId,MkMarketingChannel,Address,City,...,SyCampusID,HighestEducation,Campus,OriginalSchool,OriginalDegree,AcademicStarts,Military,SIFProgramValue,Brand,MISC_SourDes
0,62190,SF49632,AllStar Search (Bachelors),Default,ASD-10207028,0,23.0,Aggregator Plus,704 Rt 9,Keeseville,...,43,Associates,042 Purdue Global Online,UnderGradBIZ,Bachelor of Science,0,No,BSBA_HR,Aggregator Plus,AllStar Search (Bachelors)
1,61935,SF43320,TriAd (Bachelors),Default,eea49354-e001-47f3-b54b-f4317219a05a,0,1.0,Aggregator,140 warnell st,Pembroke,...,43,SomeCollegeMoreThan2,042 Purdue Global Online,HS,Bachelor of Science,0,No,BSHA,Aggregator,TriAd (Bachelors)
2,62373,SF56573,Higher Education Exclusive (Hosted),Default,NaN,0,23.0,Aggregator Plus,NaN,Del Valle,...,43,NaN,042 Purdue Global Online,GradCJ,Master of Science,0,No,MSCJ,Aggregator Plus,Higher Education Exclusive (Hosted)
3,61931,SF43316,ClassesUSA (Associates),Default,Tier2,474199205,1.0,Aggregator,18000 Kelley rd.,Fort Myers Beach,...,43,SomeCollegeLessThan2,042 Purdue Global Online,IT,Associate of Science,0,No,AASIT_NA,Aggregator,ClassesUSA (Associates)
4,61978,SF46661,Business,Default,NaN,0,23.0,Aggregator Plus,NaN,Upper Marlboro,...,43,NaN,042 Purdue Global Online,EDU,Master of Science,0,No,MATK12,Aggregator Plus,Business


In [7]:
#pip install geopy

In [138]:
#Imputing missing data in Lea
for index, row in data2.iterrows():
    if pd.isnull(row['LeadType']):
        if row['Military']=='No':
            data2.at[index, 'LeadType']='Default'
        elif row['Military']=='Yes':
            data2.at[index, 'LeadType']='Yes'  
        else:
            data2.at[index, 'LeadType']=np.NaN 
        

In [139]:
import ydata_profiling 

In [140]:
ydata_profiling.ProfileReport(data2)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [141]:
#check where all 4 i.e state,country & zip code is missing
X_missings=data2[(data2.Zip.isna()==True)&(data2.Country.isna()==True)&(data2.City.isna()==True)&(data2.State.isna()==True)]


In [142]:
#Treat Missing in State,Country
# we need atleast zip column complete filled to find geolocations , and only fewer records to be de deleted
X_data_ms=data2.drop(index=X_missings.index)


In [143]:
X_data_ms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99860 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MkVendorID            99860 non-null  int64  
 1   SourceCode            99860 non-null  object 
 2   VendorDescription     99860 non-null  object 
 3   LeadType              97807 non-null  object 
 4   VendorInternalCode    66316 non-null  object 
 5   mkCampaignID          99860 non-null  int64  
 6   mkMarketingChannelId  99860 non-null  float64
 7   MkMarketingChannel    99860 non-null  object 
 8   Address               80760 non-null  object 
 9   City                  85221 non-null  object 
 10  State                 98843 non-null  object 
 11  Country               99168 non-null  object 
 12  Zip                   99780 non-null  object 
 13  SyCampusID            99860 non-null  int64  
 14  HighestEducation      68406 non-null  object 
 15  Campus             

In [144]:
# Find missing state and country based on zip code
from geopy.geocoders import Nominatim


# Create a geocoder instance
geolocator = Nominatim(user_agent="my_app")

# Function to geocode the address based on the zip code
def geocode_address(zip_code):
    try:
        location = geolocator.geocode(zip_code, exactly_one=False, language="en", addressdetails=True)
        if location:
            return location
        else:
            return None
    except:
        return None


def get_valid_location(row, locations):
    if len(locations) > 1:
        for location in locations:
            if not location:
                return None
            if not pd.isnull(row['City']):
                if location.raw.get('address', {}).get('city') == row['City']:
                    return location
            if not pd.isnull(row['State']):
                if location.raw.get('address', {}).get('state') == row['State']:
                    return location
            if not pd.isnull(row['Country']):
                if location.raw.get('address', {}).get('country') == row['Country']:
                    return location
    else:
        locations[0]
    
# Iterate over the DataFrame and fill in missing city, state, and country values based on the zip code
for index, row in X_data_ms.iterrows():
    if pd.isnull(row['State']) or pd.isnull(row['Country']):
#     if pd.isnull(row['Country']):   
        zip_code = row['Zip']
        locations = geocode_address(zip_code)
        if locations:
            location = get_valid_location(row, locations)
            if location:
                #print(location)
#                 if pd.isnull(row['City']):
#                     X_data_ms.at[index, 'City'] = location.raw.get('address', {}).get('city')
#                 if pd.isnull(row['State']):
#                      X_data_ms.at[index, 'State'] = location.raw.get('address', {}).get('state')
                if pd.isnull(row['Country']):
                    X_data_ms.at[index, 'Country'] = location.raw.get('address', {}).get('country')

# Print the updated DataFrame
#print(X_data_ms)

In [146]:
#We can see that some missing values of city,state and country are imputed
X_data_ms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99860 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MkVendorID            99860 non-null  int64  
 1   SourceCode            99860 non-null  object 
 2   VendorDescription     99860 non-null  object 
 3   LeadType              97807 non-null  object 
 4   VendorInternalCode    66316 non-null  object 
 5   mkCampaignID          99860 non-null  int64  
 6   mkMarketingChannelId  99860 non-null  float64
 7   MkMarketingChannel    99860 non-null  object 
 8   Address               80760 non-null  object 
 9   City                  85221 non-null  object 
 10  State                 98843 non-null  object 
 11  Country               99810 non-null  object 
 12  Zip                   99780 non-null  object 
 13  SyCampusID            99860 non-null  int64  
 14  HighestEducation      68406 non-null  object 
 15  Campus             

In [259]:
X_data_ms.Country.value_counts()

United States           99200
India                      68
Canada                     51
Georgia, Republic of       44
Nigeria                    29
                        ...  
Antigua and Barbuda         1
Swaziland                   1
Gambia                      1
Saint Lucia                 1
Armenia                     1
Name: Country, Length: 136, dtype: int64

In [74]:
#locations=geolocator.geocode({'postalcode':"92705"},exactly_one=False, addressdetails=True,language='en')

In [ ]:

#Delete Address,VendorInternalCode
# if we have MkVendorID then no need of VendorDescription
# if we have mkMarketingChannelId then no need of MkMarketingChannel
# if we have SyCampusID then no need of Campus

In [147]:
#Removing columns showing high multicolinearity
X_data_df=X_data_ms.drop(columns=['MkMarketingChannel','VendorInternalCode','State','Military','OriginalSchool',
                        'Campus','HighestEducation','Address','Brand','City'])


In [149]:
final_df=X_data_df.drop_duplicates()

In [150]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97936 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MkVendorID            97936 non-null  int64  
 1   SourceCode            97936 non-null  object 
 2   VendorDescription     97936 non-null  object 
 3   LeadType              96008 non-null  object 
 4   mkCampaignID          97936 non-null  int64  
 5   mkMarketingChannelId  97936 non-null  float64
 6   Country               97886 non-null  object 
 7   Zip                   97929 non-null  object 
 8   SyCampusID            97936 non-null  int64  
 9   OriginalDegree        96054 non-null  object 
 10  AcademicStarts        97936 non-null  int64  
 11  SIFProgramValue       96207 non-null  object 
 12  MISC_SourDes          97936 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 10.5+ MB


In [151]:
final_df=final_df.fillna("Unknown")

In [152]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97936 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MkVendorID            97936 non-null  int64  
 1   SourceCode            97936 non-null  object 
 2   VendorDescription     97936 non-null  object 
 3   LeadType              97936 non-null  object 
 4   mkCampaignID          97936 non-null  int64  
 5   mkMarketingChannelId  97936 non-null  float64
 6   Country               97936 non-null  object 
 7   Zip                   97936 non-null  object 
 8   SyCampusID            97936 non-null  int64  
 9   OriginalDegree        97936 non-null  object 
 10  AcademicStarts        97936 non-null  int64  
 11  SIFProgramValue       97936 non-null  object 
 12  MISC_SourDes          97936 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 10.5+ MB


In [33]:
#pip install ydata-profiling

In [32]:
#pip install ipywidgets

In [153]:
Y_label=final_df['AcademicStarts']

In [166]:
Y_label

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: AcademicStarts, Length: 97936, dtype: int64

In [154]:
X_data=final_df[final_df.columns.difference(['AcademicStarts'])]

In [155]:
X_data

,Country,LeadType,MISC_SourDes,MkVendorID,OriginalDegree,SIFProgramValue,SourceCode,SyCampusID,VendorDescription,Zip,mkCampaignID,mkMarketingChannelId
0,United States,Default,AllStar Search (Bachelors),62190,Bachelor of Science,BSBA_HR,SF49632,43,AllStar Search (Bachelors),12944,0,23.0
1,United States,Default,TriAd (Bachelors),61935,Bachelor of Science,BSHA,SF43320,43,TriAd (Bachelors),31321,0,1.0
2,United States,Default,Higher Education Exclusive (Hosted),62373,Master of Science,MSCJ,SF56573,43,Higher Education Exclusive (Hosted),78617,0,23.0
3,United States,Default,ClassesUSA (Associates),61931,Associate of Science,AASIT_NA,SF43316,43,ClassesUSA (Associates),33931,474199205,1.0
4,United States,Default,Business,61978,Master of Science,MATK12,SF46661,43,Business,20774,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,United States,Yes,EducationDynamics Clicks Tier 1,62199,Bachelor of Science,HMD_FSEM,SF49648,43,EducationDynamics Clicks Tier 1,99703,0,23.0
99996,United States,Default,TriAd (Bachelors),61935,Bachelor of Science,BSECA,SF43320,43,TriAd (Bachelors),62449,0,1.0
99997,United States,Yes,Zeta_Email_CPC,62232,Master of Science,MSHUS,SF51880,43,Zeta_Email_CPC,84335,0,23.0
99998,United States,Default,PG_Reach Network,62012,Bachelor of Science,BSNUT,SF46073,43,PG_Reach Network,32164,0,23.0


In [194]:
categ=['MkVendorID','SourceCode','mkCampaignID','mkMarketingChannelId','Country','Zip','SyCampusID','SIFProgramValue',
       'MISC_SourDes','LeadType','OriginalDegree']

In [195]:
#labelencodiing-MkVendorID,SourceCode,mkCampaignID,mkMarketingChannelId,Country,Zip,SyCampusID,SIFProgramValue,MISC_SourDes
le = LabelEncoder()
df1 = X_data[categ].apply(le.fit_transform)


In [196]:
df1

,MkVendorID,SourceCode,mkCampaignID,mkMarketingChannelId,Country,Zip,SyCampusID,SIFProgramValue,MISC_SourDes,LeadType,OriginalDegree
0,138,272,0,14,125,1938,7,55,148,2,1
1,75,134,0,0,125,5743,7,105,536,2,1
2,256,612,0,14,125,15189,7,235,315,2,6
3,72,131,3115,0,125,6586,7,25,191,2,0
4,93,214,0,14,125,3378,7,210,166,2,6
...,...,...,...,...,...,...,...,...,...,...,...
99995,146,280,0,14,125,18460,7,184,236,10,1
99996,75,134,0,0,125,12122,7,92,536,2,1
99997,171,391,0,14,125,15961,7,250,592,10,6
99998,108,173,0,14,125,5936,7,135,462,2,1


In [188]:
#Dummy Variables-LeadType,OriginalSchool,OriginalDegree,Military,Brand
#categ1=['LeadType','OriginalDegree']

In [198]:
X_train,X_test,Y_train,Y_test=train_test_split(df1,Y_label,test_size=0.25,random_state=123)

In [199]:
Y_train.value_counts()

0    72493
1      959
Name: AcademicStarts, dtype: int64

In [200]:
#pip install imblearn

In [201]:
#since data is imbalance we will use imblearn library to balance the data
from imblearn.combine import SMOTETomek
usos=SMOTETomek(random_state=123)


In [202]:
x_train_underovr,y_train_underovr=usos.fit_resample(X_train,Y_train)

In [203]:
y_train_underovr.value_counts()

0    72141
1    72141
Name: AcademicStarts, dtype: int64

In [204]:
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=123)
x_train,y_train=ros.fit_resample(X_train,Y_train)

In [205]:
y_train.value_counts()

0    72493
1    72493
Name: AcademicStarts, dtype: int64

In [206]:
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=123)
x_train_sm,y_train_sm=sm.fit_resample(X_train,Y_train)

In [207]:
y_train_sm.value_counts()

0    72493
1    72493
Name: AcademicStarts, dtype: int64

In [208]:
from sklearn.ensemble import RandomForestClassifier

In [246]:
params={'n_estimators':[40,50,60,70,80],
       'max_features':[7,8,9,10],
       'max_depth':[20,21,22,23]}

In [247]:
model=GridSearchCV(RandomForestClassifier(random_state=123),params,cv=10,verbose=1,n_jobs=-1)
model.fit(x_train,y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'max_depth': [20, 21, 22], 'max_features': [8, 9, 10],
                         'n_estimators': [50, 60, 70, 80]},
             verbose=1)

In [248]:
model.best_params_

{'max_depth': 22, 'max_features': 8, 'n_estimators': 50}

In [249]:
model.best_score_

0.9877850210361236

In [250]:
rf=RandomForestClassifier(n_estimators=50,max_features=8,max_depth=22,verbose=1,n_jobs=-1,random_state=123)
rf.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.6s finished


RandomForestClassifier(max_depth=22, max_features=8, n_estimators=50, n_jobs=-1,
                       random_state=123, verbose=1)

In [251]:
pre_y=rf.predict_proba(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [252]:
metrics.roc_auc_score(Y_test,pre_y[:,1])

0.7779176448094142

In [258]:
metrics.accuracy_score(Y_test,rf.predict(X_test))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


0.9650792354190492